In [2]:
import pandas as pd
df=messages = pd.read_csv('SMSSpamCollection', sep='\t',
                           names=["label", "message"])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.shape

(5572, 2)

In [4]:
#independent features
X=list(df['message'])

In [5]:
y=list(df['label'])

In [6]:
y=list(pd.get_dummies(y,drop_first=True)['spam'])

In [7]:
### Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [8]:
!pip install tokenizers
!pip install --no-build-isolation --no-deps transformers

1.   call the pretrained model(Distilbert)
2.   call the tokenizer(Distilbert)


In [9]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [10]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [11]:
### Conver these encodings into Dataset objects.
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).batch(8)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(8)

In [12]:
from transformers import TFDistilBertForSequenceClassification, AutoTokenizer
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ["accuracy"]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Train the model
model.fit(train_dataset, epochs=3)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
558/558 [==============================] - 139s 199ms/step - loss: 0.0704 - accuracy: 0.9796
Epoch 2/3
558/558 [==============================] - 111s 199ms/step - loss: 0.0276 - accuracy: 0.9930
Epoch 3/3
558/558 [==============================] - 111s 199ms/step - loss: 0.0245 - accuracy: 0.9942


In [13]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


140/140 [==============================] - 12s 68ms/step - loss: 0.0244 - accuracy: 0.9946
Test Loss: 0.0244
Test Accuracy: 0.9946


In [14]:
import numpy as np

# Get model predictions
predictions = model.predict(test_dataset)

# Convert logits to class labels
predicted_labels = np.argmax(predictions.logits, axis=1)

print(predicted_labels[:10])  # Show first 10 predictions


140/140 [==============================] - 12s 67ms/step
[0 1 0 0 0 0 1 0 0 0]


In [15]:
from sklearn.metrics import classification_report

# True labels
true_labels = np.concatenate([y for x, y in test_dataset], axis=0)

# Print classification report
print(classification_report(true_labels, predicted_labels))


              precision    recall  f1-score   support

       False       0.99      1.00      1.00       955
        True       0.99      0.97      0.98       160

    accuracy                           0.99      1115
   macro avg       0.99      0.98      0.99      1115
weighted avg       0.99      0.99      0.99      1115



In [16]:
model.save("distilbert_model")

In [19]:
def predict_text(text):
    # Tokenize input
    inputs = tokenizer(text, return_tensors="tf", padding=True, truncation=True, max_length=512)

    # Get model predictions
    logits = model.predict(dict(inputs)).logits

    # Convert logits to probabilities
    probs = tf.nn.softmax(logits, axis=-1).numpy()

    # Get the predicted class
    predicted_class = tf.argmax(probs, axis=-1).numpy()[0]

    # Map class index to label
    class_labels = {0: "ham", 1: "spam"}
    predicted_label = class_labels[predicted_class]

    return predicted_label, probs


In [20]:
text = "This is an amazing product!"
predicted_class, probabilities = predict_text(text)

print(f"Predicted class: {predicted_class}")
print(f"Class probabilities: {probabilities}")


1/1 [==============================] - 0s 151ms/step
Predicted class: ham
Class probabilities: [[0.9779207 0.0220793]]


In [22]:
text = "Congratulations! You've won a $1000 gift card! 🎉 Claim your prize NOW by clicking the link below. Hurry, offer expires soon! Don't miss out on this exclusive opportunity! Click here: [FakeLink.com]"
predicted_class, probabilities = predict_text(text)

print(f"Predicted class: {predicted_class}")
print(f"Class probabilities: {probabilities}")

1/1 [==============================] - 0s 84ms/step
Predicted class: spam
Class probabilities: [[0.00159407 0.998406  ]]


In [ ]:
text = ""
predicted_class, probabilities = predict_text(text)

print(f"Predicted class: {predicted_class}")
print(f"Class probabilities: {probabilities}")